In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Importing the libraries

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from scipy import stats
import operator
#Import nltk to check english lexicon
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import (
    wordnet,
    stopwords
)

import spacy.cli
spacy.cli.download("en_core_web_md")
import spacy
nlp = spacy.load('en_core_web_md')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
# import json
# # input_file=open('drive/My Drive/CS544/nepal-quake-2015-tweets.jsonl', 'r')
# # json_decode=json.load(input_file)
# with open('drive/My Drive/CS544/nepal-quake-2015-tweets.jsonl','r') as f:
#     json_decode = json.load(f)
# result = []
# for item in json_decode:
#     my_dict={}
#     my_dict['id']=item.get('id')
#     my_dict['text']=item.get('text')
#     my_dict['retweet']=item.get('retweeted')
#     my_dict['created_at']=item.get('created_at')
    
#     result.append(my_dict)
# print(result[:5])

# Data Extraction

In [0]:
tweets_data_path = 'drive/My Drive/CS544/nepal-quake-2015-tweets.jsonl'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [14]:
print(len(tweets_data))
print(tweets_data[0])

50068
{'contributors': None, 'truncated': False, 'text': 'RT @Economynext: #SriLanka to fly emergency medical help, food to #earthquake -struck #Nepal #lka #Economynext http://t.co/t6F2RXD4tj http:…', 'is_quote_status': False, 'in_reply_to_status_id': None, 'id': 591974898139987968, 'favorite_count': 0, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'retweeted': False, 'coordinates': None, 'entities': {'symbols': [], 'user_mentions': [{'id': 2911405998, 'indices': [3, 15], 'id_str': '2911405998', 'screen_name': 'Economynext', 'name': 'EconomyNext'}], 'hashtags': [{'indices': [17, 26], 'text': 'SriLanka'}, {'indices': [66, 77], 'text': 'earthquake'}, {'indices': [86, 92], 'text': 'Nepal'}, {'indices': [93, 97], 'text': 'lka'}, {'indices': [98, 110], 'text': 'Economynext'}], 'urls': [{'url': 'http://t.co/t6F2RXD4tj', 'indices': [111, 133], 'expanded_url': 'http://www.economynext.com/Sri_Lanka_to_fly_emergency_medical_help,_food_to_earthqua

In [0]:
tweets = pd.DataFrame()
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['id'] = list(map(lambda tweet: tweet['id_str'], tweets_data))
# tweets['retweet'] = map(lambda tweet: tweet['retweeted'], tweets_data)
# tweets['created_at'] = map(lambda tweet: tweet['created_at'], tweets_data)
tweets['Timestamp'] =  list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['reply_id'] = list(map(lambda tweet: tweet['in_reply_to_status_id_str'], tweets_data))

#Create a function to see if the tweet is a retweet
def is_RT(tweet):
    if 'retweeted_status' not in tweet:
        return None      
    else:
        return tweet['retweeted_status']['id_str']
    
tweets['retweet_id'] = list(map(is_RT, tweets_data))

In [16]:
print(tweets.dtypes)
tweets['Timestamp'] =  pd.to_datetime(tweets['Timestamp'])
print(tweets.dtypes)
# tweets.sort_values(by='Timestamp', inplace=True)

text          object
id            object
Timestamp     object
reply_id      object
retweet_id    object
dtype: object
text                       object
id                         object
Timestamp     datetime64[ns, UTC]
reply_id                   object
retweet_id                 object
dtype: object


In [17]:
tweets = tweets[:1000]
tweets

,text,id,Timestamp,reply_id,retweet_id
0,RT @Economynext: #SriLanka to fly emergency me...,591974898139987968,2015-04-25 14:39:48+00:00,None,591956338080485377
1,India committed in aid to Nepal: President Pra...,591979323164233729,2015-04-25 14:57:23+00:00,None,None
2,RT @Thekkapoor: @narendramodi List of Doctor a...,591984387526234112,2015-04-25 15:17:31+00:00,None,591908885553762304
3,"Quake wake-up call for govt, need better build...",591984010714157056,2015-04-25 15:16:01+00:00,None,None
4,"#NepalEarthquake India sends 39 #NDRF team, 2 ...",591974595525128195,2015-04-25 14:38:36+00:00,None,None
5,"""@PMOIndia_RC: Indian Government is doing ever...",591983032208232448,2015-04-25 15:12:08+00:00,None,None
6,RT @ANI_news: We have to play a leading role t...,591979178733408256,2015-04-25 14:56:49+00:00,None,591903440067891200
7,RT @UNFPAasia: As emergency services step up i...,591986773212856321,2015-04-25 15:26:59+00:00,None,591908722516983809
8,1L water bottle by railways\nLocal rates from ...,591976562515316736,2015-04-25 14:46:25+00:00,None,None
9,RT @BJPLucknowBJP: Agra Medical College Buildi...,591977226586836992,2015-04-25 14:49:03+00:00,None,591970480967430144


In [18]:
tweets[:100].reply_id.value_counts()

591975170904043520    3
591978072573808640    1
591973583519354880    1
591986121086619648    1
591964478926761984    1
591979322166022144    1
591973887040061441    1
591982852780085248    1
Name: reply_id, dtype: int64

# a) Two tweets will be connected by an undirected edge if one has been posted within t time interval of the other. You may vary t from 15 to 60 minutes.

In [19]:
tweet_graph = np.zeros((1000,1000))
cnt = 0
edges = 0
for index1, row1 in tweets[:1000].iterrows(): 
#     node_list = []
    cnt+=1
    if cnt%100 == 0:
        print(cnt)
    
    for index2, row2 in tweets[index1+1:1000].iterrows(): 
#         print(index1,index2)
        minutes = (row2['Timestamp'] - row1['Timestamp'])/np.timedelta64(1,'m')
#         duration_in_s = duration.total_seconds()
#         minutes = divmod(duration_in_s, 60)[0]
#         print(minutes)
        if minutes >= 15 and minutes <= 60:
            #node_list.append(index2)
            tweet_graph[index1][index2] = 1
            tweet_graph[index2][index1] = 1
            edges += 2
            
            
#         elif minutes > 60:
#             break
#     print(node_list)
#     tweet_graph.append(node_list) 
   
print(edges)

100
200
300
400
500
600
700
800
900
1000
94928


In [20]:
G=nx.from_numpy_matrix(tweet_graph)
print(f"There are {G.number_of_nodes()} nodes and {G.number_of_edges()} edges present in the Graph")

There are 1000 nodes and 47464 edges present in the Graph


## Degree Centrality

In [21]:
degrees = [val for (node, val) in G.degree()]
print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")
print(f"The average degree of the nodes in the Graph is {np.mean(degrees):.1f}")  
print(f"The most frequent degree of the nodes found in the Graph is {stats.mode(degrees)[0][0]}")

The maximum degree of the Graph is 172
The minimum degree of the Graph is 9
The average degree of the nodes in the Graph is 94.9
The most frequent degree of the nodes found in the Graph is 31


In [22]:
if nx.is_connected(G):
    print("The graph is connected")
else:
    print("The graph is not connected")

print(f"The average clustering coefficient of the graph is {nx.average_clustering(G)}")
print(f"The transitivity of the the graph is {nx.transitivity(G)}")

The graph is connected
The average clustering coefficient of the graph is 0.2504393650281563
The transitivity of the the graph is 0.25135920858650573


In [23]:
print("Top 10 Influencers of Network Based on Degree Centrality measure\n ")
degree_centrality = nx.degree_centrality(G)
degree_centrality=sorted(degree_centrality.items(), key=operator.itemgetter(1)) 
nodes_degree=[]
for i in range(1,11):
    print("ID: %d, Degree Centrality Value : %f"%(degree_centrality[-i][0], degree_centrality[-i][1]))
    nodes_degree.append(degree_centrality[-i][0])

Top 10 Influencers of Network Based on Degree Centrality measure
 
ID: 279, Degree Centrality Value : 0.172172
ID: 269, Degree Centrality Value : 0.171171
ID: 282, Degree Centrality Value : 0.169169
ID: 272, Degree Centrality Value : 0.168168
ID: 265, Degree Centrality Value : 0.168168
ID: 458, Degree Centrality Value : 0.167167
ID: 290, Degree Centrality Value : 0.166166
ID: 259, Degree Centrality Value : 0.166166
ID: 359, Degree Centrality Value : 0.165165
ID: 293, Degree Centrality Value : 0.165165


## Closeness centrality

In [24]:
print("Top 10 Influencers of Network Based on Closeness Centrality measure\n ")
closeness_centrality = nx.closeness_centrality(G)
closeness_centrality=sorted(closeness_centrality.items(), key=operator.itemgetter(1)) 
nodes_closeness = []
for i in range(1,11):
    print("ID: %d, Closeness Centrality Value : %f"%(closeness_centrality[-i][0], closeness_centrality[-i][1]))
    nodes_closeness.append(closeness_centrality[-i][0])  

Top 10 Influencers of Network Based on Closeness Centrality measure
 
ID: 524, Closeness Centrality Value : 0.205429
ID: 520, Closeness Centrality Value : 0.205344
ID: 534, Closeness Centrality Value : 0.205260
ID: 531, Closeness Centrality Value : 0.205260
ID: 515, Closeness Centrality Value : 0.205176
ID: 512, Closeness Centrality Value : 0.205176
ID: 466, Closeness Centrality Value : 0.205176
ID: 510, Closeness Centrality Value : 0.205133
ID: 538, Closeness Centrality Value : 0.204923
ID: 499, Closeness Centrality Value : 0.204797


## Betweennes centrality

In [25]:
print("Top 10 Influencers of Network Based on Betweenness Centrality measure\n ")
betweenness_centrality = nx.betweenness_centrality(G)
betweenness_centrality=sorted(betweenness_centrality.items(), key=operator.itemgetter(1)) 
nodes_betweenness = []
for i in range(1,11):
    print("ID: %d, Betweenness Centrality Value : %f"%(betweenness_centrality[-i][0], betweenness_centrality[-i][1]))
    nodes_betweenness.append(betweenness_centrality[-i][0])  

Top 10 Influencers of Network Based on Betweenness Centrality measure
 
ID: 697, Betweenness Centrality Value : 0.171144
ID: 690, Betweenness Centrality Value : 0.161051
ID: 675, Betweenness Centrality Value : 0.158449
ID: 672, Betweenness Centrality Value : 0.154396
ID: 689, Betweenness Centrality Value : 0.116890
ID: 679, Betweenness Centrality Value : 0.081663
ID: 668, Betweenness Centrality Value : 0.072691
ID: 647, Betweenness Centrality Value : 0.067903
ID: 695, Betweenness Centrality Value : 0.063386
ID: 696, Betweenness Centrality Value : 0.060739


## Eigen vector Centrality

In [26]:
print("Top 10 Influencers of Network Based on Eigen vector Centrality measure\n ")
eigenvector_centrality = nx.eigenvector_centrality(G)
eigenvector_centrality=sorted(eigenvector_centrality.items(), key=operator.itemgetter(1)) 
nodes_eigen=[]
for i in range(1,11):
    print("ID: %d, Eigen vector Centrality Value : %f"%(eigenvector_centrality[-i][0], eigenvector_centrality[-i][1]))
    nodes_eigen.append(eigenvector_centrality[-i][0])  

Top 10 Influencers of Network Based on Eigen vector Centrality measure
 
ID: 279, Eigen vector Centrality Value : 0.067326
ID: 269, Eigen vector Centrality Value : 0.066701
ID: 282, Eigen vector Centrality Value : 0.066060
ID: 265, Eigen vector Centrality Value : 0.065558
ID: 259, Eigen vector Centrality Value : 0.065019
ID: 286, Eigen vector Centrality Value : 0.064919
ID: 272, Eigen vector Centrality Value : 0.064851
ID: 290, Eigen vector Centrality Value : 0.064519
ID: 281, Eigen vector Centrality Value : 0.064512
ID: 263, Eigen vector Centrality Value : 0.064404


## Katz Centrality

In [28]:
print("Top 10 Influencers of Network Based on Katz Centrality measure\n ")
katz_centrality = nx.katz_centrality(G,1/135.225 - 0.01)
katz_centrality=sorted(katz_centrality.items(), key=operator.itemgetter(1)) 
nodes_katz = []
for i in range(1,11):
    print("ID: %d, Katz Centrality Value : %f"%(katz_centrality[-i][0], katz_centrality[-i][1]))
    nodes_katz.append(katz_centrality[-i][0])  

Top 10 Influencers of Network Based on Katz Centrality measure
 
ID: 667, Katz Centrality Value : 0.038006
ID: 690, Katz Centrality Value : 0.037813
ID: 676, Katz Centrality Value : 0.037811
ID: 652, Katz Centrality Value : 0.037722
ID: 708, Katz Centrality Value : 0.037713
ID: 713, Katz Centrality Value : 0.037707
ID: 722, Katz Centrality Value : 0.037616
ID: 657, Katz Centrality Value : 0.037613
ID: 711, Katz Centrality Value : 0.037612
ID: 639, Katz Centrality Value : 0.037523


## Overlapping nodes based on the top 10 centrality values

In [30]:
methods = ['Degree centrality','Closeness centrality','Betweennes centrality','Eigen vector Centrality','Katz centrality']
arr = []
arr.append(nodes_degree)
arr.append(nodes_closeness)
arr.append(nodes_betweenness)
arr.append(nodes_eigen)
arr.append(nodes_katz)
for i in range(0,5):
    for j in range(i+1,5):
        print("The Common Nodes between",methods[i],"and",methods[j],set(arr[i]).intersection(arr[j]))
        print("No of nodes common between",methods[i],"and",methods[j],'are',str(len(set(arr[i]).intersection(arr[j]))),'\n')

The Common Nodes between Degree centrality and Closeness centrality set()
No of nodes common between Degree centrality and Closeness centrality are 0 

The Common Nodes between Degree centrality and Betweennes centrality set()
No of nodes common between Degree centrality and Betweennes centrality are 0 

The Common Nodes between Degree centrality and Eigen vector Centrality {290, 259, 265, 269, 272, 279, 282}
No of nodes common between Degree centrality and Eigen vector Centrality are 7 

The Common Nodes between Degree centrality and Katz centrality set()
No of nodes common between Degree centrality and Katz centrality are 0 

The Common Nodes between Closeness centrality and Betweennes centrality set()
No of nodes common between Closeness centrality and Betweennes centrality are 0 

The Common Nodes between Closeness centrality and Eigen vector Centrality set()
No of nodes common between Closeness centrality and Eigen vector Centrality are 0 

The Common Nodes between Closeness centr

# b) Edge between two tweets, if the cosine similarity between two tweet is greater than some threshold (assume 0.65), where the cosine similarity is calculated based on the similarity of the text vectors (after removing stopwords, punctuation, URLs and numbers), or one of them is a reply or retweet of the other

In [0]:
def stop_words(table):
    #We need to remove the stop words
    stop_words_list = stopwords.words('english')
    table['text'] = table['text'].str.lower()
    table['text'] = table['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words_list)]))
    return table
def preprocessing_text(table):
    #put everythin in lowercase
    table['text'] = table['text'].str.lower()
    #Replace rt indicating that was a retweet
#     table['text'] = table['text'].str.replace('rt', '')
    #Replace occurences of mentioning @UserNames
    table['text'] = table['text'].replace(r'@\w+', '', regex=True)
    #Replace links contained in the tweet
    table['text'] = table['text'].replace(r'http\S+', '', regex=True)
    table['text'] = table['text'].replace(r'www.[^ ]+', '', regex=True)
    #remove numbers
    table['text'] = table['text'].replace(r'[0-9]+', '', regex=True)
    #replace special characters and puntuation marks
    table['text'] = table['text'].replace(r'[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', regex=True)
    return table
tweets = preprocessing_text(tweets)
tweets = stop_words(tweets)

In [19]:
tweets.text[0]
print(tweets.head(5))

                                                text  ...          retweet_id
0  rt srilanka fly emergency medical help food ea...  ...  591956338080485377
1  india committed aid nepal president pranab muk...  ...                None
2           rt list doctor hospital earthquake nepal  ...  591908885553762304
3  quake wakeup call govt need better building te...  ...                None
4  nepalearthquake india sends ndrf team dogs ton...  ...                None

[5 rows x 5 columns]


In [0]:
lst1 = []
lst2 = []
for index,row in tweets.iterrows():
#     lst1.append(row['id'])
    if row['retweet_id'] is not None:
        p = int(tweets.id[tweets.id == row['retweet_id']].index[0]) if len(tweets.id[tweets.id == row['retweet_id']]) > 0 else -1 
        lst1.append(p)
    else:
        lst1.append(-1)

            
    if row['reply_id'] is not None:
        p = int(tweets.id[tweets.id == row['reply_id']].index[0]) if len(tweets.id[tweets.id == row['reply_id']]) > 0 else -1 
        lst2.append(p)
    else:
        lst2.append(-1)

# print(set(lst1).intersection(lst2))
# print(lst1,lst2)


In [0]:
tweets.retweet_id = lst1
tweets.reply_id = lst2

In [22]:
tweet_graph = np.zeros((1000,1000))
cnt = 0
edges = 0
for index1, row1 in tweets[:1000].iterrows(): 
    cnt+=1
    if cnt%100 == 0:
        print(cnt)
    doc1 = nlp(row1['text'])
    if row1['retweet_id'] != -1:
            tweet_graph[row1['retweet_id']][index1] = 1
            edges += 1
    if row1['reply_id'] != -1:
            tweet_graph[row1['reply_id']][index1] = 1
            edges += 1
    for index2, row2 in tweets[index1+1:1000].iterrows(): 
            
        doc2 = nlp(row2['text'])
        if doc1.similarity(doc2) > 0.65:
            tweet_graph[index1][index2] = 1
            tweet_graph[index2][index1] = 1
            edges += 2
        
print(edges)

100
200
300
400
500
600
700
800
900
1000
806030


In [0]:
np.save('drive/My Drive/CS544/part-2.npy',tweet_graph)

In [3]:
tweet_graph = np.load('drive/My Drive/CS544/part-2.npy')
G=nx.from_numpy_matrix(tweet_graph)
print(f"There are {G.number_of_nodes()} nodes and {G.number_of_edges()} edges present in the Graph")

There are 1000 nodes and 403014 edges present in the Graph


## Degree Centrality

In [5]:
degrees = [val for (node, val) in G.degree()]
print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")
print(f"The average degree of the nodes in the Graph is {np.mean(degrees):.1f}")  
print(f"The most frequent degree of the nodes found in the Graph is {stats.mode(degrees)[0][0]}")

The maximum degree of the Graph is 979
The minimum degree of the Graph is 13
The average degree of the nodes in the Graph is 806.0
The most frequent degree of the nodes found in the Graph is 930


In [6]:
if nx.is_connected(G):
    print("The graph is connected")
else:
    print("The graph is not connected")

print(f"The average clustering coefficient of the graph is {nx.average_clustering(G)}")
print(f"The transitivity of the the graph is {nx.transitivity(G)}")

The graph is connected
The average clustering coefficient of the graph is 0.8994451878350088
The transitivity of the the graph is 0.8895863615158284


In [7]:
print("Top 10 Influencers of Network Based on Degree Centrality measure\n ")
degree_centrality = nx.degree_centrality(G)
degree_centrality=sorted(degree_centrality.items(), key=operator.itemgetter(1)) 
nodes_degree=[]
for i in range(1,11):
    print("ID: %d, Degree Centrality Value : %f"%(degree_centrality[-i][0], degree_centrality[-i][1]))
    nodes_degree.append(degree_centrality[-i][0])

Top 10 Influencers of Network Based on Degree Centrality measure
 
ID: 816, Degree Centrality Value : 0.979980
ID: 703, Degree Centrality Value : 0.979980
ID: 691, Degree Centrality Value : 0.975976
ID: 308, Degree Centrality Value : 0.974975
ID: 780, Degree Centrality Value : 0.973974
ID: 854, Degree Centrality Value : 0.971972
ID: 178, Degree Centrality Value : 0.970971
ID: 945, Degree Centrality Value : 0.969970
ID: 873, Degree Centrality Value : 0.969970
ID: 100, Degree Centrality Value : 0.969970


## Closeness centrality

In [8]:
print("Top 10 Influencers of Network Based on Closeness Centrality measure\n ")
closeness_centrality = nx.closeness_centrality(G)
closeness_centrality=sorted(closeness_centrality.items(), key=operator.itemgetter(1)) 
nodes_closeness = []
for i in range(1,11):
    print("ID: %d, Closeness Centrality Value : %f"%(closeness_centrality[-i][0], closeness_centrality[-i][1]))
    nodes_closeness.append(closeness_centrality[-i][0])  

Top 10 Influencers of Network Based on Closeness Centrality measure
 
ID: 816, Closeness Centrality Value : 0.980373
ID: 703, Closeness Centrality Value : 0.980373
ID: 691, Closeness Centrality Value : 0.976540
ID: 308, Closeness Centrality Value : 0.975586
ID: 780, Closeness Centrality Value : 0.974634
ID: 854, Closeness Centrality Value : 0.972736
ID: 178, Closeness Centrality Value : 0.971790
ID: 945, Closeness Centrality Value : 0.970845
ID: 873, Closeness Centrality Value : 0.970845
ID: 100, Closeness Centrality Value : 0.970845


## Betweennes centrality

In [9]:
print("Top 10 Influencers of Network Based on Betweenness Centrality measure\n ")
betweenness_centrality = nx.betweenness_centrality(G)
betweenness_centrality=sorted(betweenness_centrality.items(), key=operator.itemgetter(1)) 
nodes_betweenness = []
for i in range(1,11):
    print("ID: %d, Betweenness Centrality Value : %f"%(betweenness_centrality[-i][0], betweenness_centrality[-i][1]))
    nodes_betweenness.append(betweenness_centrality[-i][0])  

Top 10 Influencers of Network Based on Betweenness Centrality measure
 
ID: 2, Betweenness Centrality Value : 0.000616
ID: 343, Betweenness Centrality Value : 0.000599
ID: 816, Betweenness Centrality Value : 0.000553
ID: 691, Betweenness Centrality Value : 0.000550
ID: 172, Betweenness Centrality Value : 0.000475
ID: 308, Betweenness Centrality Value : 0.000472
ID: 839, Betweenness Centrality Value : 0.000468
ID: 854, Betweenness Centrality Value : 0.000465
ID: 685, Betweenness Centrality Value : 0.000458
ID: 339, Betweenness Centrality Value : 0.000448


## Eigen vector Centrality

In [10]:
print("Top 10 Influencers of Network Based on Eigen vector Centrality measure\n ")
eigenvector_centrality = nx.eigenvector_centrality(G)
eigenvector_centrality=sorted(eigenvector_centrality.items(), key=operator.itemgetter(1)) 
nodes_eigen=[]
for i in range(1,11):
    print("ID: %d, Eigen vector Centrality Value : %f"%(eigenvector_centrality[-i][0], eigenvector_centrality[-i][1]))
    nodes_eigen.append(eigenvector_centrality[-i][0])  

Top 10 Influencers of Network Based on Eigen vector Centrality measure
 
ID: 703, Eigen vector Centrality Value : 0.036424
ID: 816, Eigen vector Centrality Value : 0.036351
ID: 873, Eigen vector Centrality Value : 0.036290
ID: 308, Eigen vector Centrality Value : 0.036289
ID: 780, Eigen vector Centrality Value : 0.036282
ID: 691, Eigen vector Centrality Value : 0.036251
ID: 126, Eigen vector Centrality Value : 0.036249
ID: 178, Eigen vector Centrality Value : 0.036248
ID: 299, Eigen vector Centrality Value : 0.036244
ID: 707, Eigen vector Centrality Value : 0.036230


## Katz Centrality

In [4]:
from numpy import linalg as LA
e_val,e_vec = LA.eig(tweet_graph)
print(max(e_val))

(844.8771076351193+0j)


In [0]:
print("Top 10 Influencers of Network Based on Katz Centrality measure\n ")
katz_centrality = nx.katz_centrality(G,1/844.877 - 0.01)
katz_centrality=sorted(katz_centrality.items(), key=operator.itemgetter(1)) 
nodes_katz = []
for i in range(1,11):
    print("ID: %d, Eigen vector Centrality Value : %f"%(katz_centrality[-i][0], katz_centrality[-i][1]))
    nodes_katz.append(katz_centrality[-i][0])  

## Overlapping nodes based on the top 10 centrality values

In [11]:
methods = ['Degree centrality','Closeness centrality','Betweennes centrality','Eigen vector Centrality']
arr = []
arr.append(nodes_degree)
arr.append(nodes_closeness)
arr.append(nodes_betweenness)
arr.append(nodes_eigen)

for i in range(0,4):
    for j in range(i+1,4):
        print("The Common Nodes between",methods[i],"and",methods[j],set(arr[i]).intersection(arr[j]))
        print("No of nodes common between",methods[i],"and",methods[j],'are',str(len(set(arr[i]).intersection(arr[j]))),'\n')

The Common Nodes between Degree centrality and Closeness centrality {100, 873, 780, 816, 945, 178, 691, 308, 854, 703}
No of nodes common between Degree centrality and Closeness centrality are 10 

The Common Nodes between Degree centrality and Betweennes centrality {816, 691, 308, 854}
No of nodes common between Degree centrality and Betweennes centrality are 4 

The Common Nodes between Degree centrality and Eigen vector Centrality {873, 780, 816, 178, 691, 308, 703}
No of nodes common between Degree centrality and Eigen vector Centrality are 7 

The Common Nodes between Closeness centrality and Betweennes centrality {816, 691, 308, 854}
No of nodes common between Closeness centrality and Betweennes centrality are 4 

The Common Nodes between Closeness centrality and Eigen vector Centrality {873, 780, 816, 178, 691, 308, 703}
No of nodes common between Closeness centrality and Eigen vector Centrality are 7 

The Common Nodes between Betweennes centrality and Eigen vector Centrality {

In [0]:
# import spacy
# nlp = spacy.load('en_core_web_md')
 
# doc1 = nlp(u'Hello this is document similarity calculation')
# doc2 = nlp(u'Hello this is python similarity calculation')
# doc3 = nlp(u'Hi there')
 
# print (doc1.similarity(doc2)) 
# print (doc2.similarity(doc3)) 
# print (doc1.similarity(doc3)) 